In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
)
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
FINE_TUNED_MODEL_NAME = 'model/taide_markdown_v5-full'
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_NAME)
fine_tuned_model = AutoModelForCausalLM.from_pretrained(FINE_TUNED_MODEL_NAME, load_in_8bit=True, device_map='auto')

print('load model complete')

2024-06-17 08:08:26.453980: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 08:08:26.484476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 08:08:27.124712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

load model complete


In [4]:
def get_taide_prompt(document: str, question: str, version: str = 'llama3') -> str:
    """產生TAIDE針對農業病蟲害問題的prompt。"""
    if version == "llama2":
        system_prompt = '你是一位農業病蟲害防治專家，你將看到一份markdown格式的表格，你的任務是根據此表格尋找答案並回答，回答時只需要輸出答案，不需輸出其他資訊。'
        user_message = f"問題：\n{question}\n\n表格：\n{document}\n\n"
        return f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_message} [/INST]"   
     
    elif version == "llama3":
        system_prompt = "你是一個來自台灣的AI助理，你的名字是 TAIDE，樂於以台灣人的立場幫助使用者，會用繁體中文回答問題。"
        user_message = f"你是一位農業病蟲害防治專家，你將看到一份markdown格式的表格，你的任務是根據此表格尋找答案並回答，回答時只需要輸出答案，不需輸出其他資訊。\n問題:\n{question}\n\n表格:\n{document}"
        return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    
    else:
        user_message = f"你是一位農業病蟲害防治專家，你將看到一份markdown格式的表格，你的任務是根據此表格尋找答案並回答，回答時只需要輸出答案，不需輸出其他資訊。\n問題:\n{question}\n\n表格:\n{document}"
        return user_message
    
def get_taide_reply(tokenizer, model, prompt: str) -> str:
    """使用TAIDE模型生成回答。"""
    model.eval()
    inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(device)
    generation_config = GenerationConfig(
        max_new_tokens=300,
        top_k=50,
        num_beams=2,
        num_return_sequences=1,
        early_stopping=True,
    )
    outputs = model.generate(**inputs, generation_config=generation_config)
    reply = tokenizer.batch_decode(outputs[:, inputs.input_ids.size(1) :], skip_special_tokens=True)[0]
    # reply = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return reply

### 其他 積穀害蟲 積穀害蟲的防治方法

防除方法：
（一）空倉消毒時，任選下列一種方法處理之。

| 藥劑名稱 | 每百立方公尺施藥量 | 稀釋倍數(倍) | 施藥方法 | 注意事項 |
|:--------------------------|:----------------|:---------------|:-------------|:---------------------------------------------------|
| 25%百滅寧可濕性粉劑(Permethrin) | | 100 | 以藥液噴施。 | 施藥31天後，稻穀始可出售。 |
| 10%巴賽松燻煙劑(Phoxim) | 0.5公升 | | 密閉煙燻2天。 |
| 10%陶斯松燻煙劑(Chlorpyrifos) | 0.5公升 | | 密閉煙燻2天。 | 施藥7天後，稻穀始可出售。 |
| 50%巴賽松乳劑(Phoxim) | | 40 | 以藥液噴施 |
| 50%無臭馬拉松乳劑(Malathion) | | 40 | 以藥液噴施。 |


In [5]:
title = ["其他", "積穀害蟲", "積穀害蟲的防治方法"]
document = "防除方法：\n（一）空倉消毒時，任選下列一種方法處理之。\n|藥劑名稱 | 每百立方公尺施藥量 | 稀釋倍數(倍) | 施藥方法 | 注意事項\n|:----------------------|:------------|:-----------|:-----------|:---------------------------------------------------|\n25%百滅寧可濕性粉劑(Permethrin)\t　| | 100| 以藥液噴施。| 施藥31天後，稻穀始可出售。\n10%巴賽松燻煙劑(Phoxim)| 0.5公升||  密閉煙燻2天。\t　\n10%陶斯松燻煙劑(Chlorpyrifos)| 0.5公升| | 密閉煙燻2天。| 施藥7天後，稻穀始可出售。\n50%巴賽松乳劑(Phoxim)| | 40| 以藥液噴施\n50%無臭馬拉松乳劑(Malathion)| | 40| \t以藥液噴施。"
input_context = " ".join(entity for entity in title) + "\n" + document

test_data = [
    {"question": "25%百滅寧可濕性粉劑的稀釋倍數是多少？", "answer": "100"},
    {"question": "10%巴賽松燻煙劑的施藥方法是什麼？", "answer": "密閉煙燻2天"},
    {"question": "10%陶斯松燻煙劑的施藥量？", "answer": "0.5公升"},
    {"question": "有哪些藥劑可以防治積穀害蟲？", "answer": "25%百滅寧可濕性粉劑、10%巴賽松燻煙劑、10%陶斯松燻煙劑、50%巴賽松乳劑、50%無臭馬拉松乳劑"}
]

for data in test_data:
    prompt = get_taide_prompt(input_context, data['question'], version='llama3')
    fine_tuned_answer = get_taide_reply(fine_tuned_tokenizer, fine_tuned_model, prompt)
    print(f"Question: {data['question']}")
    print(f"Answer: {data['answer']}")
    print(f"Model Reply: {fine_tuned_answer}")
    print("")

Question: 25%百滅寧可濕性粉劑的稀釋倍數是多少？
Answer: 100
Model Reply: 100。

Question: 10%巴賽松燻煙劑的施藥方法是什麼？
Answer: 密閉煙燻2天
Model Reply: 10%巴賽松燻煙劑的施藥方法是密閉煙燻2天。

Question: 10%陶斯松燻煙劑的施藥量？
Answer: 0.5公升
Model Reply: 0.5公升。

Question: 有哪些藥劑可以防治積穀害蟲？
Answer: 25%百滅寧可濕性粉劑、10%巴賽松燻煙劑、10%陶斯松燻煙劑、50%巴賽松乳劑、50%無臭馬拉松乳劑
Model Reply: 25%百滅寧可濕性粉劑、10%巴賽松燻煙劑、10%陶斯松燻煙劑、50%巴賽松乳劑、50%無臭馬拉松乳劑。

